In [1]:
import sys
import os 
sys.path.append('../src/detectron2/projects/DensePose')

In [2]:
#importing libraries
import numpy as np
from collections import Counter
import pandas as pd

import lightgbm as lgb

from sklearn.datasets import load_breast_cancer,load_boston,load_wine
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import mean_squared_error,roc_auc_score,precision_score, accuracy_score
pd.options.display.max_columns = 999
import pickle
import cv2

from skopt import BayesSearchCV

from skopt.space import Real, Categorical, Integer


from src.utils.feats import load_gei
from src.utils.results import df_results
    
import pandas as pd

import matplotlib.pyplot as plt

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



In [3]:
datapath = "../data/feats/database24_gei_480x640.pkl"

dim = (64, 48)
crop_person = True

X, y = load_gei(datapath, dim=dim, crop_person=crop_person) 

In [4]:
n_splits = 3
cv = KFold(n_splits=n_splits, random_state=42, shuffle=True)

In [5]:
# #setting up the parameters
# params={}
# params['learning_rate']=0.03
# params['boosting_type']='gbdt' #GradientBoostingDecisionTree
# params['objective']='multiclass' #Multi-class target feature
# params['metric']='multi_logloss' #metric for multi-class
# params['max_depth']=10
# params['num_class']=24 #no.of unique values in the target class not inclusive of the end value

In [6]:
# pipeline class is used as estimator to enable
# search over different model types

pipe = Pipeline([
    ('model', lgb.LGBMClassifier())
])

In [7]:
X, y = load_gei(datapath, dim=dim, crop_person=crop_person) 

lgb_search = {
    'model': Categorical([lgb.LGBMClassifier(
        application='multiclass',
        boosting_type='gbdt',
        objective='multiclass',
        metric='multi_logloss',
        num_classes=24,
        random_state=0)]),
    'model__learning_rate': Real(0.09, 0.3, prior='uniform'),
    'model__num_leaves': Integer(24, 80),
#     'model__feature_fraction': Real(0.8, 1.0, prior='uniform'),
#     'model__bagging_fraction': Real(0.8, 1.0, prior='uniform'),
#     'model__max_depth': Integer(5, 12),    
    'model__min_data_in_leaf': Integer(75, 90),  
#     'model__subsample': Real(0.01, 1.0, prior='uniform'),
#     'model__min_sum_hessian_in_leaf': Real(0.0, 100.0, prior='uniform'),    
#     'model__colsample_bytree': Real(0, 1),    
    'model__n_estimators': Integer(50, 1100),        
#     'model__min_child_samples': Integer(100, 500),        
#     'model__min_child_weight': Integer(1e-5, 1e4, prior='log-uniform'),
#     'model__reg_alpha': Integer(1e-6, 1e2, prior='log-uniform'),
#     'model__reg_lambda': Integer(1e-6, 1e2, prior='log-uniform'),    
}

In [43]:
opt = BayesSearchCV(
    pipe,
    # (parameter space, # of evaluations)    
    [(lgb_search, 1024),],
    cv=cv,
    scoring='accuracy'
)

opt.fit(X, y)

[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=84, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=84
[LightGBM] [Warning] min_data_in_leaf is set=84, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=84
[LightGBM] [Warning] min_data_in_leaf is set=84, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=84
[LightGBM] [Warning] min_data_in_leaf is set=87, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=87
[LightGBM] [Warning] min_data_in_leaf is set=87, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=87
[LightGBM] [Warning] min

[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min

[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min

[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min

[LightGBM] [Warning] min_data_in_leaf is set=86, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=86
[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min_data_in_leaf is set=86, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=86
[LightGBM] [Warning] min

...

/home/wesley.passos/miniconda3/envs/libras/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


[LightGBM] [Warning] min_data_in_leaf is set=79, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=79
[LightGBM] [Warning] min_data_in_leaf is set=79, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=79
[LightGBM] [Warning] min_data_in_leaf is set=79, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=79


/home/wesley.passos/miniconda3/envs/libras/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


[LightGBM] [Warning] min_data_in_leaf is set=79, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=79
[LightGBM] [Warning] min_data_in_leaf is set=79, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=79
[LightGBM] [Warning] min_data_in_leaf is set=79, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=79
[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min_data_in_leaf is set=80, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=80


BayesSearchCV(cv=KFold(n_splits=3, random_state=42, shuffle=True),
              estimator=Pipeline(steps=[('model', LGBMClassifier())]),
              scoring='accuracy',
              search_spaces=[({'model': Categorical(categories=(LGBMClassifier(application='multiclass', learning_rate=0.2827675596930995,
               metric='multi_logloss', min_data_in_leaf=80, n_estimators=986,
               num_classes=24, num_leaves=76, o...
               random_state=0),), prior=None),
                               'model__learning_rate': Real(low=0.09, high=0.3, prior='uniform', transform='identity'),
                               'model__min_data_in_leaf': Integer(low=75, high=90, prior='uniform', transform='identity'),
                               'model__n_estimators': Integer(low=50, high=1100, prior='uniform', transform='identity'),
                               'model__num_leaves': Integer(low=24, high=80, prior='uniform', transform='identity')},
                              1

In [44]:
df = df_results(opt)
df.to_csv('results_lightGBM.csv')
df

,model,model__learning_rate,model__min_data_in_leaf,model__n_estimators,model__num_leaves,mean_test_score,std_test_score,rank
898,"LGBMClassifier(application='multiclass', learn...",0.282343,80,975,27,0.826325,0.024194,1
285,"LGBMClassifier(application='multiclass', learn...",0.282768,80,986,76,0.826325,0.026569,1
886,"LGBMClassifier(application='multiclass', learn...",0.300000,79,50,70,0.824497,0.031832,3
792,"LGBMClassifier(application='multiclass', learn...",0.300000,79,50,71,0.824497,0.031832,3
897,"LGBMClassifier(application='multiclass', learn...",0.300000,79,50,72,0.824497,0.031832,3
...,...,...,...,...,...,...,...,...
502,"LGBMClassifier(application='multiclass', learn...",0.108520,77,60,36,0.775137,0.045914,1020
693,"LGBMClassifier(application='multiclass', learn...",0.122695,86,50,43,0.775137,0.050240,1020
237,"LGBMClassifier(application='multiclass', learn...",0.114770,84,51,80,0.769653,0.057915,1022
244,"LGBMClassifier(application='multiclass', learn...",0.092489,87,74,26,0.767824,0.068183,1023


In [17]:
# we redefine params search according to previous resuls
lgb_search = {
    'model': Categorical([lgb.LGBMClassifier(
        application='multiclass',
        boosting_type='gbdt',
        objective='multiclass',
        metric='multi_logloss',
        num_classes=24,
        random_state=0)]),
    'model__learning_rate': Real(0.2, 0.3, prior='uniform'),
    'model__num_leaves': Integer(24, 80),
#     'model__feature_fraction': Real(0.8, 1.0, prior='uniform'),
#     'model__bagging_fraction': Real(0.8, 1.0, prior='uniform'),
#     'model__max_depth': Integer(5, 12),    
    'model__min_data_in_leaf': Integer(75, 90),  
#     'model__subsample': Real(0.01, 1.0, prior='uniform'),
#     'model__min_sum_hessian_in_leaf': Real(0.0, 100.0, prior='uniform'),    
#     'model__colsample_bytree': Real(0, 1),    
    'model__n_estimators': Integer(900, 1100),        
#     'model__min_child_samples': Integer(100, 500),        
#     'model__min_child_weight': Integer(1e-5, 1e4, prior='log-uniform'),
#     'model__reg_alpha': Integer(1e-6, 1e2, prior='log-uniform'),
#     'model__reg_lambda': Integer(1e-6, 1e2, prior='log-uniform'),    
}

## lightGBM + SVD

In [8]:
from sklearn.decomposition import PCA, TruncatedSVD

In [18]:
# pipeline class is used as estimator to enable
# search over different model types

base_pipe = Pipeline([   
    ('reduction', TruncatedSVD()),
    ('model', lgb.LGBMClassifier())
])

In [19]:
X, y = load_gei(datapath, dim=dim, crop_person=crop_person) 


SVD_search = {
    'reduction': Categorical([TruncatedSVD(random_state=0),]),
    'reduction__n_components': Integer(50, 100),    
}


lgb_SVD = {
    **SVD_search,
    **lgb_search,
}

opt_SVD = BayesSearchCV(
    base_pipe,
    # (parameter space, # of evaluations)    
    [(lgb_SVD, 256),],
    cv=cv,
    scoring='accuracy'
)

opt_SVD.fit(X, y)

[LightGBM] [Warning] min_data_in_leaf is set=79, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=79
[LightGBM] [Warning] min_data_in_leaf is set=79, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=79
[LightGBM] [Warning] min_data_in_leaf is set=78, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=78
[LightGBM] [Warning] min_data_in_leaf is set=78, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=78
[LightGBM] [Warning] min_data_in_leaf is set=78, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=78
[LightGBM] [Warning] min_data_in_leaf is set=86, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=86
[LightGBM] [Warning] min_data_in_leaf is set=86, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=86
[LightGBM] [Warning] min_data_in_leaf is set=86, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=86
[LightGBM] [Warning] min

[LightGBM] [Warning] min_data_in_leaf is set=78, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=78
[LightGBM] [Warning] min_data_in_leaf is set=78, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=78
[LightGBM] [Warning] min_data_in_leaf is set=78, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=78
[LightGBM] [Warning] min_data_in_leaf is set=83, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=83
[LightGBM] [Warning] min_data_in_leaf is set=83, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=83
[LightGBM] [Warning] min_data_in_leaf is set=83, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=83
[LightGBM] [Warning] min_data_in_leaf is set=87, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=87
[LightGBM] [Warning] min_data_in_leaf is set=87, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=87
[LightGBM] [Warning] min

[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min_data_in_leaf is set=89, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=89
[LightGBM] [Warning] min_data_in_leaf is set=89, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=89
[LightGBM] [Warning] min_data_in_leaf is set=89, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=89
[LightGBM] [Warning] min_data_in_leaf is set=86, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=86
[LightGBM] [Warning] min_data_in_leaf is set=86, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=86
[LightGBM] [Warning] min_data_in_leaf is set=86, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=86
[LightGBM] [Warning] min_data_in_leaf is set=89, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=89
[LightGBM] [Warning] min

[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min

[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min

...

/home/wesley.passos/miniconda3/envs/libras/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90


/home/wesley.passos/miniconda3/envs/libras/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90


BayesSearchCV(cv=KFold(n_splits=3, random_state=42, shuffle=True),
              estimator=Pipeline(steps=[('reduction', TruncatedSVD()),
                                        ('model', LGBMClassifier())]),
              scoring='accuracy',
              search_spaces=[({'model': Categorical(categories=(LGBMClassifier(application='multiclass', learning_rate=0.22123540954694354,
               metric='multi_logloss', min_data_in_leaf=90, n_estimators=931,...
                               'model__n_estimators': Integer(low=900, high=1100, prior='uniform', transform='identity'),
                               'model__num_leaves': Integer(low=24, high=80, prior='uniform', transform='identity'),
                               'reduction': Categorical(categories=(TruncatedSVD(n_components=50, random_state=0),), prior=None),
                               'reduction__n_components': Integer(low=50, high=100, prior='uniform', transform='identity')},
                              256)])

In [20]:
df_SVD = df_results(opt_SVD)
df_SVD.to_csv('results_lightGBM_SVD.csv')
df_SVD

,model,model__learning_rate,model__min_data_in_leaf,model__n_estimators,model__num_leaves,reduction,reduction__n_components,mean_test_score,std_test_score,rank
252,"LGBMClassifier(application='multiclass', learn...",0.221235,90,931,50,"TruncatedSVD(n_components=50, random_state=0)",50,0.729433,0.024030,1
186,"LGBMClassifier(application='multiclass', learn...",0.294974,75,906,62,"TruncatedSVD(n_components=50, random_state=0)",70,0.723949,0.026408,2
114,"LGBMClassifier(application='multiclass', learn...",0.268275,90,900,45,"TruncatedSVD(n_components=50, random_state=0)",50,0.723949,0.017937,2
240,"LGBMClassifier(application='multiclass', learn...",0.212344,90,970,56,"TruncatedSVD(n_components=50, random_state=0)",50,0.723949,0.017368,2
40,"LGBMClassifier(application='multiclass', learn...",0.220338,90,938,80,"TruncatedSVD(n_components=50, random_state=0)",50,0.722121,0.013958,5
...,...,...,...,...,...,...,...,...,...,...
37,"LGBMClassifier(application='multiclass', learn...",0.298394,90,985,72,"TruncatedSVD(n_components=50, random_state=0)",82,0.685558,0.035419,252
211,"LGBMClassifier(application='multiclass', learn...",0.200000,90,901,24,"TruncatedSVD(n_components=50, random_state=0)",55,0.683729,0.032181,253
47,"LGBMClassifier(application='multiclass', learn...",0.292547,86,949,80,"TruncatedSVD(n_components=50, random_state=0)",100,0.681901,0.038969,254
76,"LGBMClassifier(application='multiclass', learn...",0.279945,75,942,32,"TruncatedSVD(n_components=50, random_state=0)",93,0.680073,0.042676,255


## lightGBM + PCA

In [21]:
X, y = load_gei(datapath, dim=dim, crop_person=crop_person) 

PCA_search = {
    'reduction': Categorical([PCA(random_state=0),]),
    'reduction__n_components': Integer(50, 80),
}

lgb_PCA = {
    **PCA_search,
    **lgb_search,
}

opt_PCA = BayesSearchCV(
    base_pipe,
    # (parameter space, # of evaluations)    
    [(lgb_PCA, 256),],
    cv=cv,
    scoring='accuracy'
)

opt_PCA.fit(X, y)

[LightGBM] [Warning] min_data_in_leaf is set=81, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=81
[LightGBM] [Warning] min_data_in_leaf is set=81, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=81
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min

[LightGBM] [Warning] min_data_in_leaf is set=82, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=82
[LightGBM] [Warning] min_data_in_leaf is set=82, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=82
[LightGBM] [Warning] min_data_in_leaf is set=82, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=82
[LightGBM] [Warning] min_data_in_leaf is set=82, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=82
[LightGBM] [Warning] min_data_in_leaf is set=82, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=82
[LightGBM] [Warning] min_data_in_leaf is set=82, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=82
[LightGBM] [Warning] min_data_in_leaf is set=82, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=82
[LightGBM] [Warning] min_data_in_leaf is set=82, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=82
[LightGBM] [Warning] min

[LightGBM] [Warning] min_data_in_leaf is set=83, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=83
[LightGBM] [Warning] min_data_in_leaf is set=81, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=81
[LightGBM] [Warning] min_data_in_leaf is set=81, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=81
[LightGBM] [Warning] min_data_in_leaf is set=81, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=81
[LightGBM] [Warning] min_data_in_leaf is set=86, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=86
[LightGBM] [Warning] min_data_in_leaf is set=86, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=86
[LightGBM] [Warning] min_data_in_leaf is set=86, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=86
[LightGBM] [Warning] min_data_in_leaf is set=83, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=83
[LightGBM] [Warning] min

[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min

[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=88, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=88
[LightGBM] [Warning] min_data_in_leaf is set=88, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=88
[LightGBM] [Warning] min

...

[LightGBM] [Warning] min_data_in_leaf is set=84, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=84
[LightGBM] [Warning] min_data_in_leaf is set=84, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=84
[LightGBM] [Warning] min_data_in_leaf is set=84, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=84
[LightGBM] [Warning] min_data_in_leaf is set=84, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=84
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min

[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min

[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=85
[LightGBM] [Warning] min

[LightGBM] [Warning] min_data_in_leaf is set=86, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=86
[LightGBM] [Warning] min_data_in_leaf is set=86, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=86
[LightGBM] [Warning] min_data_in_leaf is set=86, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=86
[LightGBM] [Warning] min_data_in_leaf is set=86, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=86
[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min

BayesSearchCV(cv=KFold(n_splits=3, random_state=42, shuffle=True),
              estimator=Pipeline(steps=[('reduction', TruncatedSVD()),
                                        ('model', LGBMClassifier())]),
              scoring='accuracy',
              search_spaces=[({'model': Categorical(categories=(LGBMClassifier(application='multiclass', learning_rate=0.21733303426999567,
               metric='multi_logloss', min_data_in_leaf=90, n_estimators=921,...
                               'model__min_data_in_leaf': Integer(low=75, high=90, prior='uniform', transform='identity'),
                               'model__n_estimators': Integer(low=900, high=1100, prior='uniform', transform='identity'),
                               'model__num_leaves': Integer(low=24, high=80, prior='uniform', transform='identity'),
                               'reduction': Categorical(categories=(PCA(n_components=71, random_state=0),), prior=None),
                               'reduction__n_componen

In [22]:
df_PCA = df_results(opt_PCA)
df_PCA.to_csv('results_lightGBM_PCA.csv')
df_PCA

,model,model__learning_rate,model__min_data_in_leaf,model__n_estimators,model__num_leaves,reduction,reduction__n_components,mean_test_score,std_test_score,rank
83,"LGBMClassifier(application='multiclass', learn...",0.217333,90,921,66,"PCA(n_components=71, random_state=0)",71,0.734918,0.014664,1
231,"LGBMClassifier(application='multiclass', learn...",0.283241,85,916,76,"PCA(n_components=71, random_state=0)",66,0.734918,0.006193,1
175,"LGBMClassifier(application='multiclass', learn...",0.227556,84,1035,53,"PCA(n_components=71, random_state=0)",66,0.731261,0.015653,3
38,"LGBMClassifier(application='multiclass', learn...",0.225568,83,917,36,"PCA(n_components=71, random_state=0)",66,0.731261,0.017307,3
141,"LGBMClassifier(application='multiclass', learn...",0.295506,85,1018,58,"PCA(n_components=71, random_state=0)",66,0.731261,0.000694,3
...,...,...,...,...,...,...,...,...,...,...
59,"LGBMClassifier(application='multiclass', learn...",0.294401,80,1090,37,"PCA(n_components=71, random_state=0)",80,0.692870,0.023465,251
201,"LGBMClassifier(application='multiclass', learn...",0.207356,90,1093,51,"PCA(n_components=71, random_state=0)",75,0.691042,0.013087,253
97,"LGBMClassifier(application='multiclass', learn...",0.203286,90,1100,64,"PCA(n_components=71, random_state=0)",72,0.689214,0.023985,254
140,"LGBMClassifier(application='multiclass', learn...",0.297709,75,900,32,"PCA(n_components=71, random_state=0)",56,0.687386,0.003799,255


## lightGBM + LDA

In [23]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [24]:
X, y = load_gei(datapath, dim=dim, crop_person=crop_person) 

LDA_search = {
    'reduction': Categorical([LDA(),]),
#     n_components cannot be larger than min(n_features, n_classes - 1).
    'reduction__n_components': Integer(15,  min(X.shape[1], np.unique(y).shape[0] - 1)),
}

lgb_LDA = {
    **LDA_search,
    **lgb_search,
}

opt_LDA = BayesSearchCV(
    base_pipe,
    # (parameter space, # of evaluations)    
    [(lgb_LDA, 256),],
    cv=cv,
    scoring='accuracy'
)

opt_LDA.fit(X, y)

[LightGBM] [Warning] min_data_in_leaf is set=81, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=81
[LightGBM] [Warning] min_data_in_leaf is set=81, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=81
[LightGBM] [Warning] min_data_in_leaf is set=82, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=82
[LightGBM] [Warning] min_data_in_leaf is set=82, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=82
[LightGBM] [Warning] min_data_in_leaf is set=82, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=82
[LightGBM] [Warning] min_data_in_leaf is set=83, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=83
[LightGBM] [Warning] min_data_in_leaf is set=83, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=83
[LightGBM] [Warning] min_data_in_leaf is set=83, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=83
[LightGBM] [Warning] min

[LightGBM] [Warning] min_data_in_leaf is set=78, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=78
[LightGBM] [Warning] min_data_in_leaf is set=78, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=78
[LightGBM] [Warning] min_data_in_leaf is set=78, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=78
[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min

[LightGBM] [Warning] min_data_in_leaf is set=90, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=90
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=86, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=86
[LightGBM] [Warning] min

[LightGBM] [Warning] min_data_in_leaf is set=83, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=83
[LightGBM] [Warning] min_data_in_leaf is set=83, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=83
[LightGBM] [Warning] min_data_in_leaf is set=81, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=81
[LightGBM] [Warning] min_data_in_leaf is set=81, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=81
[LightGBM] [Warning] min_data_in_leaf is set=81, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=81
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min

/home/wesley.passos/miniconda3/envs/libras/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


...

[LightGBM] [Warning] min_data_in_leaf is set=84, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=84
[LightGBM] [Warning] min_data_in_leaf is set=84, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=84
[LightGBM] [Warning] min_data_in_leaf is set=84, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=84
[LightGBM] [Warning] min_data_in_leaf is set=88, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=88
[LightGBM] [Warning] min_data_in_leaf is set=88, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=88
[LightGBM] [Warning] min_data_in_leaf is set=88, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=88
[LightGBM] [Warning] min_data_in_leaf is set=88, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=88
[LightGBM] [Warning] min_data_in_leaf is set=88, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=88
[LightGBM] [Warning] min

/home/wesley.passos/miniconda3/envs/libras/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


[LightGBM] [Warning] min_data_in_leaf is set=83, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=83
[LightGBM] [Warning] min_data_in_leaf is set=83, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=83
[LightGBM] [Warning] min_data_in_leaf is set=83, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=83
[LightGBM] [Warning] min_data_in_leaf is set=86, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=86
[LightGBM] [Warning] min_data_in_leaf is set=86, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=86
[LightGBM] [Warning] min_data_in_leaf is set=86, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=86
[LightGBM] [Warning] min_data_in_leaf is set=88, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=88
[LightGBM] [Warning] min_data_in_leaf is set=88, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=88
[LightGBM] [Warning] min

[LightGBM] [Warning] min_data_in_leaf is set=88, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=88
[LightGBM] [Warning] min_data_in_leaf is set=86, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=86
[LightGBM] [Warning] min_data_in_leaf is set=86, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=86
[LightGBM] [Warning] min_data_in_leaf is set=86, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=86
[LightGBM] [Warning] min_data_in_leaf is set=88, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=88
[LightGBM] [Warning] min_data_in_leaf is set=88, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=88
[LightGBM] [Warning] min_data_in_leaf is set=88, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=88
[LightGBM] [Warning] min_data_in_leaf is set=75, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=75
[LightGBM] [Warning] min

BayesSearchCV(cv=KFold(n_splits=3, random_state=42, shuffle=True),
              estimator=Pipeline(steps=[('reduction', TruncatedSVD()),
                                        ('model', LGBMClassifier())]),
              scoring='accuracy',
              search_spaces=[({'model': Categorical(categories=(LGBMClassifier(application='multiclass', learning_rate=0.2746415375075202,
               metric='multi_logloss', min_data_in_leaf=86, n_estimators=953,
               n...
                               'model__n_estimators': Integer(low=900, high=1100, prior='uniform', transform='identity'),
                               'model__num_leaves': Integer(low=24, high=80, prior='uniform', transform='identity'),
                               'reduction': Categorical(categories=(LinearDiscriminantAnalysis(n_components=23),), prior=None),
                               'reduction__n_components': Integer(low=15, high=23, prior='uniform', transform='identity')},
                             

In [26]:
df_LDA = df_results(opt_LDA)
df_LDA.to_csv('results_lightGBM_LDA.csv')
df_LDA

,model,model__learning_rate,model__min_data_in_leaf,model__n_estimators,model__num_leaves,reduction,reduction__n_components,mean_test_score,std_test_score,rank
118,"LGBMClassifier(application='multiclass', learn...",0.274642,86,953,50,LinearDiscriminantAnalysis(n_components=23),23,0.696527,0.023954,1
205,"LGBMClassifier(application='multiclass', learn...",0.291544,88,1009,36,LinearDiscriminantAnalysis(n_components=23),23,0.696527,0.022214,1
247,"LGBMClassifier(application='multiclass', learn...",0.294641,88,995,27,LinearDiscriminantAnalysis(n_components=23),23,0.694698,0.032209,3
234,"LGBMClassifier(application='multiclass', learn...",0.274104,86,955,30,LinearDiscriminantAnalysis(n_components=23),23,0.692870,0.018714,4
167,"LGBMClassifier(application='multiclass', learn...",0.292845,88,1002,25,LinearDiscriminantAnalysis(n_components=23),23,0.691042,0.019470,5
...,...,...,...,...,...,...,...,...,...,...
104,"LGBMClassifier(application='multiclass', learn...",0.203453,90,1075,79,LinearDiscriminantAnalysis(n_components=23),17,0.647166,0.013004,251
3,"LGBMClassifier(application='multiclass', learn...",0.250932,84,934,73,LinearDiscriminantAnalysis(n_components=23),15,0.647166,0.035058,251
4,"LGBMClassifier(application='multiclass', learn...",0.232821,89,1082,26,LinearDiscriminantAnalysis(n_components=23),15,0.643510,0.035869,254
73,"LGBMClassifier(application='multiclass', learn...",0.254022,90,939,78,LinearDiscriminantAnalysis(n_components=23),15,0.638026,0.042682,255
